In [47]:
import math
import torch
import torch.nn.functional as F


def fisheye_to_stereographic_torch(
    img_bgr_uint8,  # torch.uint8 [Hf, Wf, 3] 或 [Hf, Wf]
    inv_poly_param,  # list/np.array, len = N (OCamCalib inverse polynomial)
    affine_param,  # [c, d, e, xc, yc]
    W_out=1920,
    H_out=960,
    FOV_deg=140.0,  # 目标体视投影的等角视场（对角/边缘取决于你如何裁切；这里按水平半径匹配）
):
    """
    将单张鱼眼图（OCamCalib模型）重映射为体视投影（stereographic projection）。
    返回 torch.uint8 的体视投影图，形状 [H_out, W_out, 3] 或 [H_out, W_out]。

    体视投影反投影 (plane -> unit sphere) 采用从南极投影到 z=0 平面的标准形式：
        给定平面坐标 (X, Y)，球面方向向量：
            den = X^2 + Y^2 + 1
            vx = 2X / den
            vy = 2Y / den
            vz = (1 - X^2 - Y^2) / den
        与极角关系：r = sqrt(X^2 + Y^2) = 2 * tan(theta / 2)

    为了让输出图的水平半宽对应到给定 FOV/2，我们设置标定尺度 fs = (W_out/2) / r_max，
    其中 r_max = 2 * tan((FOV/2)/2)。
    """
    # ---- 1) 准备输入图像张量 ----
    if img_bgr_uint8.dim() == 2:
        Hf, Wf = img_bgr_uint8.shape
        C = 1
        img = img_bgr_uint8.unsqueeze(0).unsqueeze(0).float()  # [1,1,Hf,Wf]
    else:
        Hf, Wf, _ = img_bgr_uint8.shape
        C = 3
        img = img_bgr_uint8.permute(2, 0, 1).unsqueeze(0).float()  # [1,3,Hf,Wf]

    device = img.device
    dtype = img.dtype

    # ---- 2) 体视投影平面标定（根据期望 FOV 设置尺度）----
    # 水平半径 r_max 对应 FOV/2 ： r = 2 * tan(theta/2)
    theta_max = math.radians(FOV_deg / 2.0)
    r_max = 2.0 * math.tan(theta_max / 2.0)  # 体视投影的无量纲半径
    fs = (W_out / 2.0) / r_max  # 将无量纲半径映射到像素
    cx = W_out / 2.0
    cy = H_out / 2.0

    # ---- 3) 生成输出体视投影平面的像素网格 ----
    u = torch.linspace(0, W_out - 1, W_out, device=device, dtype=dtype)
    v = torch.linspace(0, H_out - 1, H_out, device=device, dtype=dtype)
    uu, vv = torch.meshgrid(u, v, indexing="xy")  # [H_out, W_out]

    # 平面坐标 -> 无量纲坐标 (X, Y)
    X = (uu - cx) / fs
    Y = (vv - cy) / fs

    # ---- 4) 体视投影反投影到单位球面方向向量 ----
    # 使用从南极投影的标准逆变换，使中心 (0,0) -> [0,0,1]
    r2 = X * X + Y * Y
    den = r2 + 1.0
    vx = 2.0 * X / den
    vy = 2.0 * Y / den
    vz = (1.0 - r2) / den

    # ---- 5) OCamCalib: 由方向向量求 theta，再用 inverse polynomial 得到 rho ----
    M = torch.sqrt(vx * vx + vy * vy)
    eps = 1e-8
    M_safe = torch.clamp(M, min=eps)

    # 与您原代码一致的定义（保持符号/数值兼容）
    theta = -torch.atan2(vz, M_safe)

    inv_poly = torch.tensor(inv_poly_param, dtype=dtype, device=device)  # [N]
    # Horner 法则
    rho = torch.zeros_like(theta)
    for a in reversed(inv_poly):
        rho = rho * theta + a

    # ---- 6) 将方向在鱼眼成像平面上的投影（应用 OCamCalib 的 2x2 仿射 + 主点）----
    c, d, e, xc, yc = [float(t) for t in affine_param]

    x_dir = vx / M_safe * rho
    y_dir = vy / M_safe * rho

    u_f = c * x_dir + d * y_dir + xc
    v_f = e * x_dir + y_dir + yc

    # ---- 7) grid_sample 双线性采样（对鱼眼图进行反采样）----
    grid_x = (u_f / (Wf - 1)) * 2.0 - 1.0
    grid_y = (v_f / (Hf - 1)) * 2.0 - 1.0
    grid = torch.stack([grid_x, grid_y], dim=-1).unsqueeze(0)  # [1,H_out,W_out,2]

    stereographic = F.grid_sample(
        img, grid, mode="bilinear", padding_mode="zeros", align_corners=True
    )  # [1,C,H_out,W_out]

    # ---- 8) 输出为 uint8 HxWxC ----
    stereographic = torch.clamp(stereographic, 0, 255).round().byte()
    if C == 1:
        out = stereographic.squeeze(0).squeeze(0)  # [H_out, W_out]
    else:
        out = stereographic.squeeze(0).permute(1, 2, 0)  # [H_out, W_out, 3]
    return out


# ---------------- 使用示例 ----------------
if __name__ == "__main__":
    import cv2
    import numpy as np
    import torch

    inv_poly_param = [
        889.91582358,
        528.72413142,
        -20.52846684,
        67.73730056,
        48.02888563,
        -14.14051173,
        20.22746961,
        38.15325087,
        -6.98641569,
        -26.68100659,
        -12.41222046,
        -1.83261524,
    ]
    affine_param = [
        9.99949231e-01,
        -4.44517298e-04,
        6.30993143e-04,
        9.68732891e02,
        7.48514758e02,
    ]

    # 读取 1920x1536 的鱼眼图 (BGR)
    fisheye = cv2.imread("./assets/SurCam01.jpeg", cv2.IMREAD_COLOR)
    assert fisheye is not None and fisheye.shape[1] == 1920 and fisheye.shape[0] == 1536

    img_t = torch.from_numpy(fisheye)  # [H,W,3], uint8

    W_out, H_out, FOV = 1920, 1536, 120.0
    stereo_bgr = (
        fisheye_to_stereographic_torch(
            img_t, inv_poly_param, affine_param, W_out=W_out, H_out=H_out, FOV_deg=FOV
        )
        .cpu()
        .numpy()
    )

    cv2.imwrite(f"stereographic_{W_out}x{H_out}_FOV{FOV}.png", stereo_bgr)
    print(f"saved -> stereographic_{W_out}x{H_out}_FOV{FOV}.png")

saved -> stereographic_1920x1536_FOV120.0.png


saved -> rectified_pinhole_1920x960_FOV90.png


In [26]:
import math
import torch
import torch.nn.functional as F
import numpy as np


def fisheye_to_equirectangular_torch(
    img_bgr_uint8,  # torch.uint8, [Hf, Wf, 3] 或 [Hf, Wf]
    inv_poly_param,  # OCamCalib inv_polynomial 系数列表 (θ -> ρ)
    affine_param,  # [c, d, e, xc, yc]
    We=2048,
    He=1024,  # 目标 equirectangular 分辨率，通常 He = We/2
    yaw_range=(-math.pi, math.pi),  # 经度范围（水平）：默认全 360°
    pitch_range=(-math.pi / 2, math.pi / 2),  # 纬度范围（垂直）：默认 -90°..+90°
    padding_mode="zeros",  # 超界填充：'zeros'|'border'|'reflection'
):
    """
    将单目鱼眼图（OCam 模型）重映射为 equirectangular 全景图。
    返回: torch.uint8 的全景图 [He, We, 3] 或 [He, We]。
    """
    # ---- 0) 输入张量 & 设备 ----
    if img_bgr_uint8.dim() == 2:
        Hf, Wf = img_bgr_uint8.shape
        C = 1
        img = img_bgr_uint8.unsqueeze(0).unsqueeze(0).float()  # [1,1,Hf,Wf]
    else:
        Hf, Wf, _ = img_bgr_uint8.shape
        C = 3
        img = img_bgr_uint8.permute(2, 0, 1).unsqueeze(0).float()  # [1,3,Hf,Wf]

    device = img.device
    dtype = img.dtype

    # ---- 1) 为目标 equirect 生成经纬网格 (lon, lat) ----
    # 水平 u -> 经度 lon ∈ [yaw_min, yaw_max]
    # 垂直 v -> 纬度 lat ∈ [pitch_min, pitch_max]，注意 v 向下增大，因此要“从上往下”插值
    yaw_min, yaw_max = float(yaw_range[0]), float(yaw_range[1])
    pit_min, pit_max = float(pitch_range[0]), float(pitch_range[1])

    u = torch.linspace(0, We - 1, We, device=device, dtype=dtype)
    v = torch.linspace(0, He - 1, He, device=device, dtype=dtype)
    uu, vv = torch.meshgrid(u, v, indexing="xy")  # [He, We]

    lon = yaw_min + uu * (yaw_max - yaw_min) / max(We - 1, 1)  # [-π, π]
    lat = pit_max - vv * (pit_max - pit_min) / max(He - 1, 1)  # [+π/2 .. -π/2] 从上到下

    # ---- 2) 经纬 -> 摄像机坐标系下的单位方向 (x,y,z) ----
    # 约定：相机坐标 z 前方、x 右、y 下（与像素 v 同向）
    # 用这样的定义：前方 (lon=0, lat=0) -> (0,0,1)
    # 顶部 (lat=+90°)        -> (0,-1,0)
    # 右侧 (lon=+90°, lat=0) -> (1,0,0)
    cos_lat = torch.cos(lat)
    sin_lat = torch.sin(lat)
    sin_lon = torch.sin(lon)
    cos_lon = torch.cos(lon)

    dx = cos_lat * sin_lon
    dy = -sin_lat  # 注意负号：lat 向上为正，而像素 y 向下为正
    dz = cos_lat * cos_lon

    # ---- 3) OCamCalib 投影：方向 -> 鱼眼像素 (u_f, v_f) ----
    eps = 1e-6
    M = torch.sqrt(dx * dx + dy * dy)
    M_safe = torch.clamp(M, min=eps)
    theta = -torch.atan2(
        dz, M_safe
    )  # 与你给的函数一致：theta = -atan2(z, sqrt(x^2+y^2))

    # ρ(θ) 用 Horner 法稳定计算
    inv_poly = torch.tensor(inv_poly_param, dtype=dtype, device=device)
    rho = torch.zeros_like(theta)
    for a in reversed(inv_poly):
        rho = rho * theta + a

    # 单位化平面方向 * ρ
    x_dir = dx / M_safe * rho
    y_dir = dy / M_safe * rho

    # 仿射 + 主点
    c, d, e, xc, yc = [float(t) for t in affine_param]
    u_f = c * x_dir + d * y_dir + xc
    v_f = e * x_dir + y_dir + yc

    # ---- 4) grid_sample 采样需要把 (u_f, v_f) 归一化到 [-1,1] ----
    grid_x = (u_f / (Wf - 1)) * 2.0 - 1.0
    grid_y = (v_f / (Hf - 1)) * 2.0 - 1.0
    grid = torch.stack([grid_x, grid_y], dim=-1).unsqueeze(0)  # [1,He,We,2]

    # ---- 5) 从鱼眼图采样得到全景 ----
    pano = F.grid_sample(
        img, grid, mode="bilinear", padding_mode=padding_mode, align_corners=True
    )  # [1,C,He,We]

    # ---- 6) 输出 uint8 HxWxC ----
    pano = torch.clamp(pano, 0, 255).round().byte()
    if C == 1:
        return pano.squeeze(0).squeeze(0)  # [He, We]
    else:
        return pano.squeeze(0).permute(1, 2, 0)  # [He, We, 3]


# ---------------- 使用示例 ----------------
if __name__ == "__main__":
    import cv2

    inv_poly_param = [
        889.91582358,
        528.72413142,
        -20.52846684,
        67.73730056,
        48.02888563,
        -14.14051173,
        20.22746961,
        38.15325087,
        -6.98641569,
        -26.68100659,
        -12.41222046,
        -1.83261524,
    ]
    affine_param = [
        9.99949231e-01,
        -4.44517298e-04,
        6.30993143e-04,
        9.68732891e02,
        7.48514758e02,
    ]

    # 读入你的 1920x1536 鱼眼图（BGR）
    fisheye = cv2.imread("./assets/SurCam01.jpeg", cv2.IMREAD_COLOR)
    assert fisheye is not None and fisheye.shape[1] == 1920 and fisheye.shape[0] == 1536

    # -> torch
    img_t = torch.from_numpy(fisheye)  # [Hf,Wf,3], uint8

    # 生成 360x180 的 equirect（注意：背面会是黑的，因为单目看不到）
    pano = (
        fisheye_to_equirectangular_torch(
            img_t,
            inv_poly_param,
            affine_param,
            We=2048,
            He=1024,  # 典型比例：He = We/2
            yaw_range=(-math.pi, math.pi),  # 全 360°
            pitch_range=(-math.pi / 2, math.pi / 2),  # -90°..+90°
            padding_mode="zeros",
        )
        .cpu()
        .numpy()
    )

    cv2.imwrite("pano_equirect_2048x1024.png", pano)
    print("saved -> pano_equirect_2048x1024.png")

    # 如果你只想要前半球（避免黑边），把 yaw 限制到 180°：
    pano_front = (
        fisheye_to_equirectangular_torch(
            img_t,
            inv_poly_param,
            affine_param,
            We=2048,
            He=1024,
            yaw_range=(-math.pi / 2, math.pi / 2),  # 只取 -90°..+90° 水平
            pitch_range=(-math.pi / 2, math.pi / 2),  # 垂直仍是 -90°..+90°
        )
        .cpu()
        .numpy()
    )
    cv2.imwrite("pano_front_hemi_2048x1024.png", pano_front)
    print("saved -> pano_front_hemi_2048x1024.png")

saved -> pano_equirect_2048x1024.png
saved -> pano_front_hemi_2048x1024.png
